# Model Deployment with Kubernetes and ONNX

This notebook covers deploying ML models using Kubernetes, building upon concepts from:
- Module 5 - FastAPI deployment
- Module 9 - Serverless and ONNX

Instead of deploying to AWS Lambda, we'll deploy to a local Kubernetes cluster using Kind (Kubernetes in Docker), making it accessible and cost-free for learners.

**Video**: https://www.youtube.com/live/c_CzCsCnWoU?si=fgQ56JwunM3NoiWm

## Prerequisites

Before starting, ensure you have:
- Docker installed and running
- Basic understanding of Python and ML models
- Familiarity with FastAPI

## Part 1: Python Environment Setup

Set up the Python environment first so you can use the correct kernel for this notebook.

### Step 1.1: Install uv

uv is a fast Python package installer and resolver.

In [1]:
%pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 72.0 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!uv --version

uv 0.9.16


### Step 1.2: Create Python Environment

Create a new Python environment for week-10 using uv.


In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop
[ ! -d ".venv" ] && uv venv
uv sync
.venv/bin/python --version

Using existing .venv


Resolved 1 package in 0.63ms
Uninstalled 29 packages in 128ms
 - asttokens==3.0.1
 - comm==0.2.3
 - debugpy==1.8.17
 - decorator==5.2.1
 - executing==2.2.1
 - ipykernel==7.1.0
 - ipython==9.8.0
 - ipython-pygments-lexers==1.1.1
 - jedi==0.19.2
 - jupyter-client==8.6.3
 - jupyter-core==5.9.1
 - matplotlib-inline==0.2.1
 - nest-asyncio==1.6.0
 - packaging==25.0
 - parso==0.8.5
 - pexpect==4.9.0
 - platformdirs==4.5.1
 - prompt-toolkit==3.0.52
 - psutil==7.1.3
 - ptyprocess==0.7.0
 - pure-eval==0.2.3
 - pygments==2.19.2
 - python-dateutil==2.9.0.post0
 - pyzmq==27.1.0
 - six==1.17.0
 - stack-data==0.6.3
 - tornado==6.5.2
 - traitlets==5.14.3
 - wcwidth==0.2.14


Python 3.13.9


### Step 1.3: Register Environment as Jupyter Kernel

Register the environment as a Jupyter kernel so you can use it in this notebook.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop
uv pip install ipykernel
.venv/bin/python -m ipykernel install --user --name week-10 --display-name "week-10"


Resolved 34 packages in 165ms
 Downloaded debugpy
 Downloaded pygments
 Downloaded jedi
Prepared 27 packages in 481ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 29 packages in 192ms
 + asttokens==3.0.1
 + comm==0.2.3
 + debugpy==1.8.17
 + decorator==5.2.1
 + executing==2.2.1
 + ipykernel==7.1.0
 + ipython==9.8.0
 + ipython-pygments-lexers==1.1.1
 + jedi==0.19.2
 + jupyter-client==8.6.3
 + jupyter-core==5.9.1
 + matplotlib-inline==0.2.1
 + nest-asyncio==1.6.0
 + packaging==25.0
 + parso==0.8.5
 + pexpect==4.9.0
 + platformdirs==4.5.1
 + prompt-toolkit==3.0.52
 + psutil==7.1.3
 + ptyprocess==0.7.0
 + pure-eval==0.2.3
 + pygments==2.19.2
 + python-dateutil==2.9.0.post0
 + pyzmq==27.1.0
 + six==1.17.0
 + stack-data==0.6.3
 + tornado==6.5.2
 + traitlets==5.14.3
 + wcwidth==0.2.14


Installed kernelspec week-10 in /home/codespace/.local/share/jupyter/kernels/week-10
Kernel registered. Refresh Jupyter to see it.


## Part 2: Kubernetes Tools Setup

Install Kubernetes command-line tools and set up a local cluster.


### Step 2.1: Install kubectl

kubectl is the Kubernetes command-line tool.

**Note**: Installation commands vary by platform:
- **Windows (with Chocolatey)**: `choco install kubernetes-cli`
- **macOS**: `brew install kubectl`
- **Linux**: See commands below

In [1]:
%%bash
mkdir -p ~/bin
cd ~/bin
curl -sLO "https://dl.k8s.io/release/$(curl -Ls https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
chmod +x kubectl
export PATH="${PATH}:${HOME}/bin"

In [2]:
!export PATH="${PATH}:${HOME}/bin" && kubectl version --client

Client Version: v1.33.2
Kustomize Version: v5.6.0


### Step 2.2: Install Kind

Kind (Kubernetes in Docker) allows you to run Kubernetes clusters locally using Docker containers.

**Note**: Installation commands vary by platform:
- **Windows (with Chocolatey)**: `choco install kind`
- **macOS**: `brew install kind`
- **Linux**: See commands below


In [3]:
%%bash
mkdir -p ~/bin
curl -sLo ~/bin/kind https://kind.sigs.k8s.io/dl/v0.20.0/kind-linux-amd64
chmod +x ~/bin/kind
export PATH="${PATH}:${HOME}/bin"


### Step 2.3: Create Kind Cluster

Create a local Kubernetes cluster. This will:
- Create a single-node Kubernetes cluster (default Kind cluster: 1 control plane node and 0 worker node; you can check by running `kubectl get nodes`) 
- Configure kubectl to use this cluster
- Take a few minutes on first run (downloads images)


In [4]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kind create cluster --name mlzoomcamp 2>&1 || true


Creating cluster "mlzoomcamp" ...
 • Ensuring node image (kindest/node:v1.27.3) 🖼  ...
 ✓ Ensuring node image (kindest/node:v1.27.3) 🖼
 • Preparing nodes 📦   ...
 ✓ Preparing nodes 📦 
 • Writing configuration 📜  ...
 ✓ Writing configuration 📜
 • Starting control-plane 🕹️  ...
 ✓ Starting control-plane 🕹️
 • Installing CNI 🔌  ...
 ✓ Installing CNI 🔌
 • Installing StorageClass 💾  ...
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-mlzoomcamp"
You can now use your cluster with:

kubectl cluster-info --context kind-mlzoomcamp

Thanks for using kind! 😊


## Part 3: Verification

Verify that everything is set up correctly.

In [6]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl cluster-info
kubectl get nodes


Kubernetes control plane is running at https://127.0.0.1:35743
CoreDNS is running at https://127.0.0.1:35743/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
NAME                       STATUS   ROLES           AGE    VERSION
mlzoomcamp-control-plane   Ready    control-plane   2m7s   v1.27.3


## Part 4: Model Preparation

Download the pre-trained ONNX model for clothing classification.

Pre-trained PyTorch model converted to ONNX format. Predicts one of 10 clothing categories from an image URL.


In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
wget -q https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/dl-models/clothing_classifier_mobilenet_v2_latest.onnx -O clothing-model.onnx
ls -lh clothing-model.onnx

## Part 5: Building the FastAPI Service

Create a FastAPI application that serves the ONNX model for inference.

### Step 5.1: Initialize Project

Initialize the service project and add dependencies.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
[ ! -f pyproject.toml ] && uv init
rm -f main.py


In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
rm -f uv.lock
uv add fastapi uvicorn onnxruntime keras-image-helper numpy


### Step 5.2: FastAPI Application

Create the FastAPI application that uses ONNX Runtime for inference and includes health checks for Kubernetes.

### Create FastAPI Application

Create the FastAPI application file.

In [ ]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/service/app.py
import numpy as np
import onnxruntime as ort
from keras_image_helper import create_preprocessor
from fastapi import FastAPI
from pydantic import BaseModel, HttpUrl
import uvicorn

app = FastAPI(title="clothing-classifier")

def preprocess_pytorch_style(X):
    X = X / 255.0
    
    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)
    
    X = X.transpose(0, 3, 1, 2)
    X = (X - mean) / std
    
    return X.astype(np.float32)

preprocessor = create_preprocessor(
    preprocess_pytorch_style,
    target_size=(224, 224)
)

# Load the ONNX model using ONNX Runtime
session = ort.InferenceSession(
    "clothing-model.onnx",
    providers=["CPUExecutionProvider"]  # Run on CPU
)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

classes = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]


# Pydantic model for incoming request
# Ensures "url" is a valid URL string
class PredictRequest(BaseModel):
    url: HttpUrl


# Pydantic response model
# Ensures API returns structured, validated JSON
class PredictResponse(BaseModel):
    predictions: dict[str, float]
    top_class: str
    top_probability: float


def predict(url: str):
    X = preprocessor.from_url(url)
    result = session.run([output_name], {input_name: X})
    float_predictions = result[0][0].tolist()
    predictions_dict = dict(zip(classes, float_predictions))
    
    top_class = max(predictions_dict, key=predictions_dict.get)
    top_probability = predictions_dict[top_class]
    
    return predictions_dict, top_class, top_probability


@app.get("/")
def root():
    return {"message": "Clothing Classification Service"}

# Health check endpoint - for monitoring / Kubernetes probes
@app.get("/health")
def health():
    return {"status": "healthy"}

# Main prediction API endpoint
# - Accepts JSON body with image URL
# - Returns probabilities and top class
@app.post("/predict", response_model=PredictResponse)
def predict_endpoint(request: PredictRequest):
    predictions, top_class, top_prob = predict(str(request.url))
    
    return PredictResponse(
        predictions=predictions,
        top_class=top_class,
        top_probability=top_prob
    )


if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8080)

### Create Test Script

Create a test script to verify the service works.

In [ ]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/service/test.py
import requests

url = 'http://localhost:8080/predict'

request = {
    "url": "http://bit.ly/mlbookcamp-pants"
}

response = requests.post(url, json=request)
result = response.json()

print(f"Top prediction: {result['top_class']} ({result['top_probability']:.2%})")
print(f"\nAll predictions:")
for cls, prob in result['predictions'].items():
    print(f"  {cls:12s}: {prob:.2%}")


### Step 5.3: Test Locally

Run the service and test it.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
uv run uvicorn app:app --host 0.0.0.0 --port 8080 --reload

**Note:** Start the service above, then run the test below in a separate terminal or after stopping the service.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
uv run python test.py

## Part 6: Docker Containerization

Containerize the application with Docker.

### Step 6.1: Create Dockerfile

Create a Dockerfile to containerize the service.

In [2]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/service/Dockerfile
FROM python:3.13.5-slim-bookworm

COPY --from=ghcr.io/astral-sh/uv:latest /uv /uvx /bin/

WORKDIR /app

ENV PATH="/app/.venv/bin:$PATH"

COPY pyproject.toml README.md .python-version ./
RUN uv sync

COPY app.py clothing-model.onnx ./

EXPOSE 8080

ENTRYPOINT ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8080"]


Overwriting /workspaces/ml-zoomcamp-works/week-10/workshop/service/Dockerfile


### Step 6.2: Build Docker Image

Build the Docker image.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
docker build -t clothing-classifier:v1 .


### Step 6.3: Test Container

Run the container and test it.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
docker run -d --rm -p 8080:8080 --name clothing-classifier clothing-classifier:v1
sleep 3
curl -s http://localhost:8080/health


In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
uv run python test.py
docker stop clothing-classifier


## Part 7: Loading Image to Kind

Load the Docker image into the Kind cluster.

Kind clusters run in Docker, so they can't access images from your local Docker daemon by default. Load the image into Kind.

In [1]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kind load docker-image clothing-classifier:v1 --name mlzoomcamp


Image: "clothing-classifier:v1" with ID "sha256:28de0f9e811509097ed088f3be093dc9f218a9e391d3fbc4b9e342137a7e91ee" not yet present on node "mlzoomcamp-control-plane", loading...


## Part 8: Kubernetes Deployment

Deploy the service to Kubernetes.

### Understanding Kubernetes Resources

- **Pod**: The smallest deployable unit in Kubernetes (one or more containers)
- **Deployment**: Manages a set of identical Pods, handles updates and scaling
- **Service**: Exposes Pods to network traffic
- **HPA (Horizontal Pod Autoscaler)**: Automatically scales Pods based on metrics

### Step 8.1: Create Deployment

Create the Deployment manifest and deploy it.

In [2]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/deployment.yaml

apiVersion: apps/v1
kind: Deployment

metadata:
  name: clothing-classifier              # Name of the deployment
  labels:
    app: clothing-classifier

spec:
  replicas: 2                            # Run 2 identical pods
  selector:
    matchLabels:
      app: clothing-classifier           # Pods must have this label

  template:
    metadata:
      labels:
        app: clothing-classifier         # Label added to each pod

    spec:
      containers:
      - name: clothing-classifier
        image: clothing-classifier:v1    # Docker image to run
        imagePullPolicy: Never           # Use local image (Kind)

        ports:
        - containerPort: 8080            # App listens on port 8080

        resources:
          requests:                      # Minimum resources reserved
            memory: "256Mi"
            cpu: "250m"
          limits:                        # Maximum allowed resources
            memory: "512Mi"
            cpu: "500m"

        # Kubernetes restarts the pod if this probe fails
        livenessProbe:
          httpGet:
            path: /health
            port: 8080
          initialDelaySeconds: 10
          periodSeconds: 10

        # Pod receives traffic only when this probe succeeds
        readinessProbe:
          httpGet:
            path: /health
            port: 8080
          initialDelaySeconds: 5
          periodSeconds: 5

Overwriting /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/deployment.yaml


NOTE:

| Setting                         | Meaning                                                 |
| ------------------------------- | ------------------------------------------------------- |
| `imagePullPolicy: Always`       | Always pull from registry → fails in Kind               |
| `imagePullPolicy: IfNotPresent` | Pull only if missing locally                            |
| `imagePullPolicy: Never`        | **Never pull; only use local image** → correct for Kind |


Let's deploy the deployment.

In [3]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl apply -f /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/deployment.yaml

deployment.apps/clothing-classifier created


Check the deployment:

In [4]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl get deployments
kubectl get pods


NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
clothing-classifier   2/2     2            2           15s
NAME                                   READY   STATUS    RESTARTS   AGE
clothing-classifier-6fbf5bc4d4-cbmf5   1/1     Running   0          15s
clothing-classifier-6fbf5bc4d4-dvb7r   1/1     Running   0          15s


NOTE:

| Column         | Explanation                                                      |
| -------------- | ---------------------------------------------------------------- |
| **NAME**       | Name of your Deployment (`clothing-classifier`)                  |
| **READY**      | `2/2` → 2 pods are running and ready (matching desired replicas) |
| **UP-TO-DATE** | 2 pods have the latest version of your Deployment spec           |
| **AVAILABLE**  | 2 pods are available to serve traffic                            |
| **AGE**        | Deployment was created 15 seconds ago                            |


### Step 8.2: Create Service

Create the Service manifest and deploy it.

In [5]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/service.yaml

apiVersion: v1
kind: Service

metadata:
  name: clothing-classifier          # Service name

spec:
  type: NodePort                     # Expose service on a port of each node
  selector:
    app: clothing-classifier         # Route traffic to pods with this label

  ports:
  - protocol: TCP
    port: 8080                       # Service port (inside cluster)
    targetPort: 8080                 # Container port
    nodePort: 30080                  # Port exposed externally on the node

Overwriting /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/service.yaml


Let's deploy the service.

In [6]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl apply -f /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/service.yaml


service/clothing-classifier created


Check the service:

In [7]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl get services


NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
clothing-classifier   NodePort    10.96.174.215   <none>        8080:30080/TCP   28s
kubernetes            ClusterIP   10.96.0.1       <none>        443/TCP          112m


NOTE:

| Column          | Meaning                                                              | Example from Your Output                 |
| --------------- | -------------------------------------------------------------------- | ---------------------------------------- |
| **NAME**        | The name of the Service                                              | `clothing-classifier`                    |
| **TYPE**        | How the Service is exposed (`ClusterIP`, `NodePort`, `LoadBalancer`) | `NodePort` → reachable through node’s IP |
| **CLUSTER-IP**  | Internal virtual IP used *inside the cluster*                        | `10.96.174.215`                          |
| **EXTERNAL-IP** | Public IP (for cloud LoadBalancer) or `<none>` if not applicable     | `<none>` (expected for NodePort)         |
| **PORT(S)**     | `<service-port>:<node-port>/protocol`                                | `8080:30080/TCP`                         |


See also [Understanding Kubernetes Services Through the Nobi House Analogy](docs/Understanding_Kubernetes_Server.md).

### Step 8.3: Test Deployed Service

Use port-forward to access the service (NodePort doesn't work with Kind).

If you try

```
curl http://localhost:30080/health
```

This assumes that **NodePort works** in your Kubernetes cluster and that port **30080** is reachable from your machine.

But in Kind, NodePort does *not* work the way you expect. Kind runs Kubernetes **inside Docker containers**.
So each Kubernetes “node” is actually a Docker container and NodePorts open ports **inside the node container**, not on your host machine. That means:

* `localhost:30080` on your laptop ≠ port 30080 inside the Kind node
* Therefore your curl request fails

NodePort *can* work in Kind, but only if the cluster is created with special port mappings.

Since NodePort is not accessible from outside Kind, we “tunnel” traffic using port-forward:

```
kubectl port-forward service/clothing-classifier 30080:8080
```

This means:

| Local machine       | Port-forward | Pod            |
| ------------------- | ------------ | -------------- |
| `localhost:30080` → | forwarded to | `service:8080` |

So Kubernetes **forwards** local traffic to the service inside the cluster.

In [10]:
%%bash
export PATH="${PATH}:${HOME}/bin"

# Stop any existing port-forward process for this service (if running)
# pkill exits with error if no process exists, so "|| true" avoids breaking the script
pkill -f 'kubectl port-forward.*clothing-classifier' || true
sleep 1

# Start a new port-forward in the background
# - Forwards localhost:30080 → service/clothing-classifier:8080
# - Redirects output to /dev/null to keep the notebook clean
kubectl port-forward service/clothing-classifier 30080:8080 > /dev/null 2>&1 &
sleep 2
curl -s http://localhost:30080/health


{"status":"healthy"}

## Part 9: Horizontal Pod Autoscaling

Automatically scale Pods based on CPU usage.

### Step 9.1: Install Metrics Server

First, we need metrics-server for HPA to work. Install it in kubectl:

In [2]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml


serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


NOTE: See [Metrics Server (Kubernetes)](docs/Metrics_Server_(Kubernetes).md) for more details.

For Kind, we need to patch metrics-server to work without TLS. See [Why Metrics Server Needs a Patch in Kind](docs/Metrics_Server_Patch_in_Kind.md).

In [3]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl patch -n kube-system deployment metrics-server --type=json -p '[{"op":"add","path":"/spec/template/spec/containers/0/args/-","value":"--kubelet-insecure-tls"}]'


deployment.apps/metrics-server patched


Wait for metrics-server to be ready:

In [4]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl get deployment metrics-server -n kube-system


NAME             READY   UP-TO-DATE   AVAILABLE   AGE
metrics-server   1/1     1            1           26s


### Step 9.2: Create HPA

Create and deploy the Horizontal Pod Autoscaler.

In [5]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/hpa.yaml

apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler

metadata:
  name: clothing-classifier-hpa         # Name of the HPA resource

spec:
  scaleTargetRef:                       # The workload the HPA will scale
    apiVersion: apps/v1
    kind: Deployment
    name: clothing-classifier           # Scale this Deployment's pods

  minReplicas: 2                        # Never scale below 2 pods
  maxReplicas: 10                       # Upper limit: up to 10 pods

  metrics:
  - type: Resource                      # Use resource-based metrics (CPU)
    resource:
      name: cpu                         # Monitor CPU usage
      target:
        type: Utilization               # Target CPU % of each pod
        averageUtilization: 50          # Scale up if avg CPU > 50%

Overwriting /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/hpa.yaml


Deploy HPA:

In [6]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl apply -f /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/hpa.yaml


horizontalpodautoscaler.autoscaling/clothing-classifier-hpa created


Check HPA status:

In [8]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl get hpa

NAME                      REFERENCE                        TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
clothing-classifier-hpa   Deployment/clothing-classifier   0%/50%    2         10        2          16s


### Step 9.3: Test Autoscaling

Generate load to trigger autoscaling.

In [9]:
%%writefile /workspaces/ml-zoomcamp-works/week-10/workshop/load_test.py
import time
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Endpoint exposed via port-forward or NodePort
url = 'http://localhost:30080/predict'

# Payload sent with each request
request_data = {
    "url": "http://bit.ly/mlbookcamp-pants"
}

# Function executed by each worker thread
def send_request(_):
    try:
        response = requests.post(url, json=request_data, timeout=5)
        return response.status_code
    except Exception as e:
        return f"Error: {e}"

print("Starting load test...")
print("Watch HPA with: kubectl get hpa -w")
print("Watch pods with: kubectl get pods -w")

num_requests = 1000          # Total number of requests to send
concurrent_workers = 50      # Number of threads sending requests in parallel

print(f"Sending {num_requests} requests with {concurrent_workers} concurrent workers")
print()

start_time = time.time()

# Launch concurrent requests using a thread pool
with ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
    futures = [executor.submit(send_request, i) for i in range(num_requests)]
    results = [future.result() for future in as_completed(futures)]

end_time = time.time()
duration = end_time - start_time

# Count how many requests succeeded (HTTP 200)
success_count = sum(1 for r in results if r == 200)
error_count = len(results) - success_count

print("Load test complete!")
print(f"Duration: {duration:.2f} seconds")
print(f"Requests per second: {len(results)/duration:.2f}")
print(f"Successful requests: {success_count}")
print(f"Failed requests: {error_count}")
print()

Overwriting /workspaces/ml-zoomcamp-works/week-10/workshop/load_test.py


First, check that you can access the endpoint:

In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"

# Stop any existing port-forward process for this service (if running)
# pkill exits with error if no process exists, so "|| true" avoids breaking the script
pkill -f 'kubectl port-forward.*clothing-classifier' || true
sleep 1

# Start a new port-forward in the background
# - Forwards localhost:30080 → service/clothing-classifier:8080
# - Redirects output to /dev/null to keep the notebook clean
kubectl port-forward service/clothing-classifier 30080:8080 > /dev/null 2>&1 &
sleep 2
curl -s http://localhost:30080/health


Run the test:

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop
uv add requests
uv run python load_test.py


**Note:** While the load test runs, watch HPA and pods in separate terminals:
- `kubectl get hpa -w`
- `kubectl get pods -w`

## Part 10: Managing the Deployment

Common operations for managing the Kubernetes deployment.

### Step 10.1: Update Application

Rebuild and update the deployment with a new image version.

In [ ]:
%%bash
cd /workspaces/ml-zoomcamp-works/week-10/workshop/service
docker build -t clothing-classifier:v2 .
export PATH="${PATH}:${HOME}/bin"
kind load docker-image clothing-classifier:v2 --name mlzoomcamp


In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl set image deployment/clothing-classifier clothing-classifier=clothing-classifier:v2


In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl rollout status deployment/clothing-classifier


### Step 10.2: Manual Scaling

Manually scale the deployment.

In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl scale deployment clothing-classifier --replicas=5
kubectl get pods


### Step 10.3: View Logs

View logs from the deployment.

In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl logs -l app=clothing-classifier --tail=20


### Step 10.4: Debugging

Debug deployment issues.

In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl describe deployment clothing-classifier


In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl get events --sort-by='.lastTimestamp' | tail -10


## Part 11: Cleanup

Clean up Docker images and Kubernetes resources.

### Step 11.1: Cleanup Kubernetes Resources

Delete deployments, services, and HPA.

In [10]:
%%bash
export PATH="${PATH}:${HOME}/bin"
kubectl delete -f /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/deployment.yaml
kubectl delete -f /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/service.yaml
kubectl delete -f /workspaces/ml-zoomcamp-works/week-10/workshop/k8s/hpa.yaml


deployment.apps "clothing-classifier" deleted from default namespace
service "clothing-classifier" deleted from default namespace
horizontalpodautoscaler.autoscaling "clothing-classifier-hpa" deleted from default namespace


Stop any existing `kubectl port-forward` process for the `clothing-classifier` service so a new one can start without conflicts:


In [12]:
%%bash
export PATH="${PATH}:${HOME}/bin"
pkill -f 'kubectl port-forward.*clothing-classifier' || true


### Step 11.2: Cleanup Docker Images

Remove Docker images.

In [13]:
%%bash
docker rmi clothing-classifier:v1 clothing-classifier:v2 || true


Untagged: clothing-classifier:v1
Deleted: sha256:28de0f9e811509097ed088f3be093dc9f218a9e391d3fbc4b9e342137a7e91ee


Error response from daemon: No such image: clothing-classifier:v2


### Step 11.3: Cleanup Kind Cluster (Optional)

Delete the Kind cluster if you want to start fresh.

In [ ]:
%%bash
export PATH="${PATH}:${HOME}/bin"
# Uncomment the line below to delete the cluster:
# kind delete cluster --name mlzoomcamp
